# Basic HuggingFace Model Deployment on Amazon SageMaker

## Introduction

This notebook walks you through deploying "distilbert-base-uncased-finetuned-sst-2-english" model on CPU instances and inf1 instances. This particular notebook was tested with PyTorch==2.0.0 and Python-3.9 on a Ubuntu 20.04 container. Bear in mind the kernel you choose for running the container isn't necessarily the same for training and inference. The notebook, training and inference containers are separate and are separately deployed. You can change the Kernel from top right corner of this notebook

## Install specific versions for some libraries. 

This is important because SageMaker SDK is updated routinely and sometimes only certain versions are supported. e.g. in this case sagemaker==2.173.0 (latest) supports transformers==4.28.1 wheras the latest one available (at the time of authorship) is 4.31.0

In [ ]:
!pip install sagemaker==2.173.0 transformers==4.28.1 accelerate pip

## Import the modules

In [ ]:
import sagemaker
import transformers
import torch

In [ ]:
# check their versions just to be sure; If version mismatch and doesn't correct itself after reinstalling, best to start a new kernel.

print(f"SageMaker API Version: {sagemaker.__version__}")
print(f"Transformers API Version: {transformers.__version__}")
print(f"Torch Version: {torch.__version__}")

## Create the session here

This is the Session helper class that brings things like IAM Execution role and region and other things into the notebook. Having the right region and s3 context is important because not all resources are available in all regions. 

In [ ]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

## HuggingFace setup

This is where we simply use the HuggingFaceModel class to pull pre-trained models from HuggingFace 🤗

This class expects a few things

1. Environment - HuggingFace uses ModelID and related Pipelines. The Pipeline defines the task for the model. e.g. sentiment-analysis, summarization etc. All the [tasks can be found here](https://huggingface.co/docs/transformers/v4.31.0/en/main_classes/pipelines#transformers.pipeline.task)
2. Role - it expects proper permissions. You can just use the default execution role in this case. This is defined when you create a domain and username in the SageMaker console
3. container parameters - it either expects "Python version, PyTorch Version" OR just the "Deep Learning Container Image URI"

Find the corresponding [Deep Learning image you would like to use here](https://github.com/aws/deep-learning-containers/blob/master/available_images.md)

In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel

These images are hosted in a public ECR repository which still requries permissions and auth. The credentials to access these ECR repos are already in the context of this container. But, it's for only a specific region you're opearting in. In this case it's us-east-2. But we'll use the session object's boto3 region.

In [ ]:
image_uri = f"763104351884.dkr.ecr.{sess.boto_region_name}.amazonaws.com/huggingface-pytorch-inference:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04"

In [ ]:
MODEL_ID = "distilbert-base-uncased-finetuned-sst-2-english"
MODEL_TASK = "text-classification"
INSTANCE_TYPE = 'ml.g5.xlarge' # Change this according to your preference
INSTANCE_COUNT = 1

In [ ]:
hf_env = {
       'HF_MODEL_ID': MODEL_ID,
       'HF_TASK': MODEL_TASK       
    }

In [ ]:
huggingface_model = HuggingFaceModel(
    env=hf_env,
    role=role, # iam role with permissions to create an Endpoint
    image_uri=image_uri # uri for the deep learning container 
)

In [ ]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=INSTANCE_COUNT,
   instance_type=INSTANCE_TYPE,
)

### Make predictions

In [ ]:
data={"inputs": "it 's a charming and often affecting journey ."}

In [ ]:
predictor.predict(data)

In [ ]:
predictor.delete_endpoint()

In [ ]:
# Metrics

In [ ]:
# send 10000 requests
for i in range(10000):
    resp = predictor.predict(
        data={"inputs": "it 's a charming and often affecting journey ."}
    )

In [ ]:
print(f"https://console.aws.amazon.com/cloudwatch/home?region={sess.boto_region_name}#metricsV2:graph=~(metrics~(~(~'AWS*2fSageMaker~'ModelLatency~'EndpointName~'{predictor.endpoint_name}~'VariantName~'AllTraffic))~view~'timeSeries~stacked~false~region~'{sess.boto_region_name}~start~'-PT5M~end~'P0D~stat~'Average~period~30);query=~'*7bAWS*2fSageMaker*2cEndpointName*2cVariantName*7d*20{predictor.endpoint_name}")

## Let's talk Amazon Silicon

In [ ]:
!pip config set global.extra-index-url https://pip.repos.neuron.amazonaws.com

In [ ]:
!pip install torch-neuron neuron-cc[tensorflow]

In [ ]:
import os
import tensorflow  # to workaround a protobuf version conflict issue
import torch
import torch.neuron
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, torchscript=True)

# create dummy input for max length 128
dummy_input = "dummy input which will be padded later"
max_length = 128
embeddings = tokenizer(dummy_input, max_length=max_length, padding="max_length",return_tensors="pt")
neuron_inputs = tuple(embeddings.values())

# compile model with torch.neuron.trace and update config
model_neuron = torch.neuron.trace(model, neuron_inputs)
model.config.update({"traced_sequence_length": max_length})

# save tokenizer, neuron model and config for later use
save_dir="tmp"
os.makedirs("tmp",exist_ok=True)
model_neuron.save(os.path.join(save_dir,"neuron_model.pt"))
tokenizer.save_pretrained(save_dir)
model.config.save_pretrained(save_dir)